In [1]:
using Revise

In [2]:
using ModelVerification

In [3]:
using LazySets
using PyCall
using CSV
using ONNX
using Flux
using Test
using NNlib
using ONNXNaiveNASflux
using NaiveNASflux
using DataStructures
# using DataFrames
# import Flux: flatten

In [28]:
# using Flux: onehotbatch, onecold, flatten
# using Flux.Losses: logitcrossentropy
# using Statistics: mean
using CUDA
using MLDatasets: CIFAR10, MNIST
using MLUtils: splitobs, DataLoader
using Accessors
using Profile
using LinearAlgebra

In [5]:
#= abstract type Perturbation end

mutable struct LP{T<:Real} <: Perturbation
    norm::Float64
    eps::Float64
end
 =#
mutable struct CrownBound3{T<:Real}
    batch_Low::AbstractArray   
    batch_Up::AbstractArray          
end

In [6]:
#= function init_perturbation(node, batch_input, perturbation_info::LP, batch_info, global_info)#batch_input include data, data_min, data_max
        if(perturbation_info.norm == Inf)
            batch_Low = batch_input .- perturbation_info["eps"]
            batch_Up = batch_input .+ perturbation_info["eps"] 
        else
            batch_Low = batch_input
            batch_Up = batch_input
        end
        new_bound = CrownBound2(batch_Low, batch_Up) #batch_info[node]["data_min"], batch_info[node]["data_max"])
end =#

function init_perturbation(node, batch_input, perturbation_info, batch_info, global_info)#batch_input include data, data_min, data_max
    if(perturbation_info["norm"] == Inf)
        batch_Low = batch_input .- perturbation_info["eps"]
        batch_Up = batch_input .+ perturbation_info["eps"] 
    else
        batch_Low = batch_input
        batch_Up = batch_input
    end
    println(size(batch_Low))
    println(size(batch_Up))
    new_bound = CrownBound3(batch_Low, batch_Up) #batch_info[node]["data_min"], batch_info[node]["data_max"])
end

init_perturbation (generic function with 1 method)

In [7]:
onnx_model_path = "/home/verification/ModelVerification.jl/debug.onnx"

comp_graph = ONNXNaiveNASflux.load(onnx_model_path, infer_shapes=false)
batch_info = Dict()
global_info = Dict()
push!(global_info, "activation_number" => 0)
push!(global_info, "activation_nodes" => [])
push!(global_info, "final_nodes" => [])
push!(global_info, "all_nodes" => [])
for (index, vertex) in enumerate(ONNXNaiveNASflux.vertices(comp_graph))
    if index == 1 # the vertex which index == 1 has no useful information, so it's output node will be the start node of the model
        push!(global_info, "start_nodes" => [NaiveNASflux.name(output_node) for output_node in outputs(vertex)]) 
        continue
    end 
        
    node_name = NaiveNASflux.name(vertex)
    new_dict = Dict() # store the information of this vertex 
    push!(new_dict, "vertex" => vertex)
    push!(new_dict, "layer" => NaiveNASflux.layer(vertex))
    push!(new_dict, "index" => index)
    push!(new_dict, "outputs" => [NaiveNASflux.name(output_node) for output_node in outputs(vertex)])
    # add input nodes of current node. If the input nodes of current node have activation(except identity), then the "inputs" should be the activation node
    if !(node_name in global_info["start_nodes"])# if current node is not one of the start node
        push!(new_dict, "inputs" => [])
        for input_node in inputs(vertex)
            input_node_name = NaiveNASflux.name(input_node)
            if hasfield(typeof(batch_info[input_node_name]["layer"]), :σ) && string(batch_info[input_node_name]["layer"].σ) != "identity"
                push!(new_dict["inputs"], batch_info[input_node_name]["outputs"][1])
            else
                push!(new_dict["inputs"], input_node_name)
            end
        end
    else
        push!(new_dict, "inputs" => nothing)
    end
    
    if length(string(NaiveNASflux.name(vertex))) >= 7 && string(NaiveNASflux.name(vertex))[1:7] == "Flatten" 
        push!(new_dict, "layer" => Flux.flatten)
        push!(batch_info, node_name => new_dict) #new_dict belongs to batch_info
        push!(global_info["all_nodes"], node_name) 
    elseif length(string(NaiveNASflux.name(vertex))) >= 3 && string(NaiveNASflux.name(vertex))[1:3] == "add" 
        push!(new_dict, "layer" => +)
        push!(batch_info, node_name => new_dict) #new_dict belongs to batch_info
        push!(global_info["all_nodes"], node_name) 
    elseif length(string(NaiveNASflux.name(vertex))) >= 4 && string(NaiveNASflux.name(vertex))[1:4] == "relu" 
        global_info["activation_number"] += 1
        node_name = "relu" * "_" * string(global_info["activation_number"]) #activate == "relu_5" doesn't mean this node is 5th relu node, but means this node is 5th activation node
        push!(new_dict, "layer" => NNlib.relu)
        push!(batch_info, node_name => new_dict) #new_dict belongs to batch_info
        push!(global_info["activation_nodes"], node_name)
        push!(global_info["all_nodes"], node_name) 
    elseif hasfield(typeof(NaiveNASflux.layer(vertex)), :σ) && string(NaiveNASflux.layer(vertex).σ) != "identity"#split this layer into a linear layer and a activative layer
        global_info["activation_number"] += 1
        activation_name = string(NaiveNASflux.layer(vertex).σ) * "_" * string(global_info["activation_number"])
        push!(new_dict, "outputs" => [activation_name]) #new_dict store the information of the linear layer
        push!(batch_info, node_name => new_dict) #new_dict belongs to batch_info
        push!(global_info["all_nodes"], node_name) 
            
        activation_new_dict = Dict()#store the information of the activative layer
        push!(activation_new_dict, "vertex" => vertex)
        push!(activation_new_dict, "layer" => NaiveNASflux.layer(vertex).σ)
        push!(activation_new_dict, "index" => index)# Do not need to change index
        push!(activation_new_dict, "inputs" => [node_name])
        push!(activation_new_dict, "outputs" => [NaiveNASflux.name(output_nodes) for output_nodes in outputs(vertex)])
        push!(batch_info, activation_name => activation_new_dict)
        push!(global_info["activation_nodes"], activation_name)
        push!(global_info["all_nodes"], activation_name) 

        node_name = activation_name #for getting the final_nodes
    else
        push!(batch_info, node_name => new_dict) #new_dict belongs to batch_info
        push!(global_info["all_nodes"], node_name) 
    end
    
    if length(batch_info[node_name]["outputs"]) == 0  #the final_node node has no output nodes
        push!(global_info["final_nodes"], node_name) 
    end
end


Set(Any["

Flatten", "Relu", "Gemm"])


In [8]:
for node in global_info["all_nodes"]
    # Check whether all prior intermediate bounds already exist
    push!(batch_info[node], "prior_checked" => false)
    # check whether weights are perturbed and set nonlinear for some operations
    if isa(batch_info[node]["layer"], Flux.Dense) || isa(batch_info[node]["layer"], Flux.Conv) || isa(batch_info[node]["layer"], Flux.BatchNorm)#if the params of Linear, Conv, Batchnorm need to be perturbed, the Linear, Conv, Batchnorm will be non_linear
        push!(batch_info[node], "nonlinear" => false)
        if haskey(batch_info[node], "weight_ptb") || haskey(batch_info[node], "bias_ptb" )
            push!(batch_info[node], "nonlinear" => true)
        end
    end
end

In [25]:
input = ones(28, 28, 1, 1) .* 0013
push!(global_info, "model_inputs" => input)
push!(batch_info["Flatten_0"], "perturbation_info" => Dict("norm" => Inf, "eps" => 0013))
push!(batch_info["relu_1"], "requires_input_bounds" => [1])
#push!(batch_info["Flatten_0"], "weight_ptb" => true)
for node in global_info["all_nodes"]
    # Check whether all prior intermediate bounds already exist
    push!(batch_info[node], "prior_checked" => false)
    push!(batch_info[node], "used" => true)
    push!(batch_info[node], "ptb" => true)
    push!(batch_info[node], "alpha" => 1.0)
    push!(batch_info[node], "beta" => 1.0)
    push!(batch_info[node], "weight_ptb" => false)
    push!(batch_info[node], "bias_ptb" => false)
end
println(isa(batch_info["dense_0"]["layer"], Flux.Dense))
lower = input .- 0013
upper = input .+ 0013
push!(batch_info["Flatten_0"], "interval" => [lower, upper])
push!(batch_info["Flatten_0"], "lower" => lower)
push!(batch_info["Flatten_0"], "upper" => upper)
push!(batch_info["Flatten_0"], "center" => input)
push!(batch_info["dense_0"], "center" => input)


true


Dict{Any, Any} with 17 entries:
  "vertex"        => MutationVertex(CompVertex(LazyMutable(MutableLayer(Dense(7…
  "ptb"           => true
  "outputs"       => ["relu_1"]
  "layer"         => Dense(784 => 200, relu)
  "prior_checked" => false
  "bounded"       => true
  "nonlinear"     => false
  "weight_ptb"    => false
  "alpha"         => 1.0
  "used"          => true
  "center"        => [0.013 0.013 … 0.013 0.013; 0.013 0.013 … 0.013 0.013; … ;…
  "index"         => 3
  "beta"          => 1.0
  "lA"            => [1, 2, 3, 4, 5, 6, 8, 9, 11, 12  …  188, 190, 192, 193, 19…
  "uA"            => [1, 2, 3, 4, 5, 6, 8, 9, 11, 12  …  188, 190, 192, 193, 19…
  "bias_ptb"      => false
  "inputs"        => Any["Flatten_0"]

In [10]:
for i in batch_info
    println(typeof(i))
end
println(global_info["all_nodes"])

Pair{Any, Any}
Pair{Any, Any}
Pair{Any, Any}
Pair{Any, Any}


Any["Flatten_0", "dense_0", "relu_1", "dense_1"]


In [11]:
function check_prior_bounds(node, batch_info, global_info)
    if batch_info[node]["prior_checked"] || !(batch_info[node]["used"] && batch_info[node]["ptb"])
        return
    end
    
    if !isnothing(batch_info[node]["inputs"])
        for input_node in batch_info[node]["inputs"]
            check_prior_bounds(input_node, batch_info, global_info)
        end
    end

    if haskey(batch_info[node], "nonlinear") && batch_info[node]["nonlinear"]
        for input_node in batch_info[node]["inputs"]
            #compute_intermediate_bounds(input_node, batch_info, global_info, prior_checked = true)
        end
    end

    if haskey(batch_info[node], "requires_input_bounds")
        for i in batch_info[node]["requires_input_bounds"]
            #compute_intermediate_bounds(batch_info[node]["inputs"][i], batch_info, global_info, prior_checked = true)
        end
    end
    push!(batch_info[node], "prior_checked" => true)
end

check_prior_bounds (generic function with 1 method)

In [12]:
final_node = global_info["final_nodes"][1]
check_prior_bounds(final_node, batch_info, global_info)

Dict{Any, Any} with 13 entries:
  "vertex"        => MutationVertex(CompVertex(LazyMutable(MutableLayer(Dense(2…
  "ptb"           => true
  "outputs"       => Any[]
  "layer"         => Dense(200 => 10)
  "prior_checked" => true
  "nonlinear"     => false
  "weight_ptb"    => false
  "alpha"         => 1.0
  "used"          => true
  "index"         => 4
  "beta"          => 1.0
  "bias_ptb"      => false
  "inputs"        => Any["relu_1"]

In [32]:
a = ones(1, 155, 784) 
b = ones(1, 784, 1)
d = ones(1, 5) .* (-1)
#c = a * b

[1.0 1.0 1.0 1.0 1.0]


In [77]:
push!(global_info, "bound_lower" => true)
push!(global_info, "bound_upper" => true)
function get_degrees(node, batch_info, global_info)
    degrees = Dict()
    push!(batch_info[node], "bounded" => false)
    queue = Queue{Any}()
    enqueue!(queue, node)
    while !isempty(queue)
        node = dequeue!(queue)
        if !isnothing(batch_info[node]["inputs"])
            for input_node in batch_info[node]["inputs"]
                if haskey(degrees, input_node)
                    push!(degrees, input_node => degrees[input_node] + 1)
                else
                    push!(degrees, input_node => 1)
                end
                if batch_info[input_node]["bounded"]
                    push!(batch_info[input_node], "bounded" => false)
                    enqueue!(queue, input_node)
                end
            end
        end
    end
    return degrees
end

function is_activation(l)
    for f in NNlib.ACTIVATIONS
        isa(l, typeof(@eval NNlib.$(f))) && return true
    end
    return false
end

function _preprocess(node, batch_info, global_info, a, b, c = nothing)#a:input node's lower/upper b:weight's lower/upper c:bias's lower/upper
    if batch_info[node]["alpha"] != 1.0 
        a = batch_info[node]["alpha"] .* a
    end
    if !isnothing(c)
        if batch_info[node]["beta"] != 1.0 
            c = batch_info[node]["beta"] .* c
        end
    end
    return a, b, c
end

function bound_oneside(last_A, weight, bias)
    if isnothing(last_A)
        return nothing, 0
    end

    weight = reshape(weight, (size(weight)..., 1)) 
    weight = repeat(weight, 1, 1, size(last_A)[end]) #add batch dim in weight
    weight = permutedims(weight, (2, 1, 3)) #permute the 1st and 2sd dims for batched_mul
    new_A = NNlib.batched_mul(weight, last_A) #note: must be weight * last_A, not last_A * weight
    
    if !isnothing(bias)
        bias = reshape(bias, (size(bias)..., 1, 1)) #add input dim in weight
        bias = repeat(bias, 1, 1, size(last_A)[end]) #add batch dim in weight
        bias = permutedims(bias, (2, 1, 3))
        sum_bias = NNlib.batched_mul(bias, last_A)
    else
        sum_bias = 0.0
    end

    return next_A, sum_bias
end

function bound_backward(layer::Dense, node, batch_info, global_info)
    last_lA = batch_info[node]["lA"] #last_lA means lA that has already stored in batch_info[node]
    last_uA = batch_info[node]["uA"] #last_lA means lA that has already stored in batch_info[node]
    input_node = batch_info[node]["inputs"][1] #Dense layer could only have 1 input Node
    if haskey(batch_info[input_node], "lower") 
        input_node_lb = batch_info[input_node]["lower"]
    else
        input_node_lb = nothing
    end

    if haskey(batch_info[input_node], "upper") 
        input_node_ub = batch_info[input_node]["upper"]
    else
        input_node_ub = nothing
    end

    #TO DO: we haven't consider the perturbation in weight and bias
    input_node_lb, weight_lb, bias_lb = _preprocess(node, batch_info, global_info, input_node_lb, layer.weight, layer.bias)
    input_node_ub, weight_ub, bias_ub = _preprocess(node, batch_info, global_info, input_node_ub, layer.weight, layer.bias)
    lA_y = uA_y = lA_bias = uA_bias = nothing
    lbias = ubias = 0
    batch_size = !isnothing(last_lA) ? size(last_lA)[end] : size(last_lA)[end]

    if !batch_info[node]["weight_ptb"] && (!batch_info[node]["bias_ptb"] || isnothing(layer.bias))
        weight = weight_lb
        bias = bias_lb
        
        #= index = last_lA
        coeffs = nothing
        
        if !isnothing(weight)
            new_weight = weight[index, :] #get the parameters that correspond to unstable neuron
            lA_x = reshape(new_weight, (size(new_weight)..., 1))
        end
        if !isnothing(bias)
            new_bias = bias[index, :] #get the parameters that correspond to unstable neuron
            lbias = reshape(new_bias, (size(new_bias)..., 1))
        end
        uA_x, ubias = lA_x, lbias =#
        
        lA_x, lbias = bound_oneside(last_lA, weight, bias)
        uA_x, ubias = bound_oneside(last_uA, weight, bias)

        return [(lA_x, uA_x), (lA_y, uA_y), (lA_bias, uA_bias)], lbias, ubias
    end

    return input_node_lb, weight_lb, bias_lb
end


function add_bound(node, input_node, lA, uA, batch_info, global_info)
    if !isnothing(lA)
        if isnothing(batch_info[input_node]["lA"])
            # First A added to this node.
            push!(batch_info[input_node], "lA" => lA)
        else
            #node_pre.zero_lA_mtx = node_pre.zero_lA_mtx and node.zero_backward_coeffs_l
            new_node_lA = batch_info[input_node]["lA"] .+ lA
            push!(batch_info[input_node], "lA" => new_node_lA)
        end
    end
    if !isnothing(uA)
        if isnothing(batch_info[input_node]["uA"])
            # First A added to this node.
            push!(batch_info[input_node], "uA" => uA)
        else
            #node_pre.zero_lA_mtx = node_pre.zero_lA_mtx and node.zero_backward_coeffs_l
            new_node_uA = batch_info[input_node]["uA"] .+ uA
            push!(batch_info[input_node], "uA" => new_node_uA)
        end
    end
end

function concretize_matrix(x, A, perturbation_info, sign, batch_info, global_info)
    x_L = x - perturbation_info["eps"]
    x_U = x + perturbation_info["eps"]
    center = (x_L .+ x_U) ./ 2.0
    diff = (x_U .- x_L) ./ 2.0
    bound = NNlib.batched_mul(A, center) .+ sign .* NNlib.batched_mul(abs.(A), diff)
    return bound
end

function ptb_concretize(x, A, sign, batch_info, global_info)
    if isnothing(A)
        return nothing
    end
    return concretize_matrix(x, A, sign, batch_info, global_info)
end

function concretize(lb, ub, batch_info, global_info)
    node = global_info["start_nodes"][1]
    if haskey(batch_info[node], "perturbation_info") #the node need to be perturbated
        if global_info["bound_lower"]
            lb = lb .+ ptb_concretize(global_info["model_inputs"], batch_info[node]["lA"], -1, batch_info, global_info)
        else
            lb = nothing
        end
        if global_info["bound_upper"]
            ub = ub .+ ptb_concretize(model_inputs, batch_info[node]["uA"], +1)
        else
            ub = nothing
        end    
    else #the node doesn't need to be perturbated
    end
end

#TO Do
function preprocess_C(C, node, batch_info, global_info)
    batch_size = size(C)[end]
    output_dim = size(C)[2]
    output_shape = [-1]
    return C, batch_size, output_dim, output_shape
end

function backward_general(C, node, unstable_idx, unstable_size, batch_info, global_info)
    for node in global_info["all_nodes"]
        push!(batch_info[node], "lA" => nothing)
        push!(batch_info[node], "uA" => nothing)
        push!(batch_info[node], "bounded" => true)
    end
    
    push!(batch_info[node], "lA" => global_info["bound_lower"] ? C : nothing)
    push!(batch_info[node], "uA" => global_info["bound_upper"] ? C : nothing)
    lb = ub = 0
    degree_out = get_degrees(node, batch_info, global_info)
    C, batch_size, output_dim, output_shape = preprocess_C(C, node, batch_info, global_info)#size(C)=(10, 9, 1) 
    #batch_size = 1, output_dim = 9, output_shape = [-1] 
    queue = Queue{Any}()
    enqueue!(queue, node)

    while !isempty(queue)
        n = dequeue!(queue)
        push!(batch_info[n], "bounded" => true)

        for input_node in batch_info[n]["inputs"]
            degree_out[input_node] -= 1
            if degree_out[input_node] == 0
                enqueue!(queue, input_node)
            end
        end

        if !batch_info[n]["ptb"]
            if !haskey(batch_info[n], "forward_value")
                get_forward_value(n)
            end
            lb, ub = add_constant_node(lb, ub, n, batch_info, global_info)
            continue
        end
        
        if isa(typeof(batch_info[n]["layer"]), typeof(relu))
            A, lower_b, upper_b = bound_backward(batch_info[n]["layer"], n, node, unstable_idx)
        elseif is_activation(batch_info[n]["layer"])   
        else
            A, lower_b, upper_b = bound_backward(batch_info[n]["layer"], node, batch_info, global_info) 
        end

        lb = lb .+ lower_b
        ub = ub .+ upper_b

        for (i, input_node) in enumerate(batch_info[node]["inputs"])
            add_bound(node, input_node, A[i][1], A[i][2], batch_info, global_info)
        end
    end

    lb, ub = concretize(lb, ub, batch_info, global_info)

end

unstable_idx = [0,    1,   2,   3,   4,   5,   7,   8,  10,  11,  12,  13,  15,  16, 19,  20,  21,  22,  23,  24,  25,  26,  28,  29,  30,  31,  32,  35,
37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  51, 52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  64,  65,  68,
70,  71,  73,  74,  75,  77,  78,  80,  82,  84,  86,  87,  89,  90, 91,  92,  93,  94,  95,  96,  98,  99, 100, 102, 103, 104, 105, 106,
107, 108, 109, 110, 112, 113, 114, 115, 116, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132, 136, 137, 138, 140, 141, 142, 143,
145, 146, 148, 149, 151, 153, 154, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 171, 172, 173, 174, 176, 177, 178, 179, 180,
181, 182, 184, 185, 186, 187, 189, 191, 192, 194, 195, 196, 197, 198, 199]
unstable_idx = unstable_idx .+ 1
unstable_size = 155
C = [-1 0 0 0 0 0 0 0 0; 
0 -1 0 0 0 0 0 0 0; 
0 0 -1 0 0 0 0 0 0; 
0 0 0 -1 0 0 0 0 0; 
0 0 0 0 -1 0 0 0 0; 
0 0 0 0 0 -1 0 0 0; 
1 1 1 1 1 1 1 1 1; 
0 0 0 0 0 0 -1 0 0; 
0 0 0 0 0 0 0 -1 0; 
0 0 0 0 0 0 0 0 -1;;;] # size(C) = (10, 9, 1) = (class_number, spec_number, batch_size)

backward_general(C, "dense_1", unstable_idx, unstable_size, batch_info, global_info)

(10,)
(1, 10, 1)
[0.8996035499999999 -0.42670631 0.45987493 -1.42302948 -1.31039339 -0.31737727 1.7427944599999998 0.92044079 1.6577974;;;]


MethodError: MethodError: no method matching iterate(::Nothing)
Closest candidates are:
  iterate(!Matched::Union{LinRange, StepRangeLen}) at range.jl:872
  iterate(!Matched::Union{LinRange, StepRangeLen}, !Matched::Integer) at range.jl:872
  iterate(!Matched::T) where T<:Union{Base.KeySet{<:Any, <:Dict}, Base.ValueIterator{<:Dict}} at dict.jl:712
  ...

In [14]:
function compute_intermediate_bounds(node, batch_info, global_info, prior_checked = false)
    if haskey(batch_info[node], "lower")# && !isnothing(batch_info[node]["lower"])
        return
    end

    if !prior_checked
        check_prior_bounds(node, batch_info, global_info)
    end

    if !batch_info[node]["ptb"]
        fv = get_forward_value(node)
        push!(batch_info[node], "interval" => [fv, fv])
        push!(batch_info[node], "lower" => fv)
        push!(batch_info[node], "upper" => fv)
        return
    end
end

compute_intermediate_bounds (generic function with 2 methods)

In [ ]:
batch_size = size(global_info["model_inputs"])[end]
dim_in = 0
for node in global_info["start_nodes"]
    value = global_info["model_inputs"]
    if haskey(batch_info[node], "ptb") 
        ret_init = init_perturbation(node, value, batch_info[node]["perturbation_info"], batch_info, global_info)
        push!(batch_info[node], "interval" => [ret_init.batch_Low, ret_init.batch_Up])
        push!(batch_info[node], "lower" => ret_init.batch_Low)
        push!(batch_info[node], "upper" => ret_init.batch_Up)
        push!(batch_info[node], "bound" => ret_init)
    else
        # This input/parameter does not have perturbation.
        push!(batch_info[node], "interval" => [value, value])
        push!(batch_info[node], "forward_value" => value)
        new_bound = CrownBound3(value, value)#, batch_info[node]["data_min", batch_info[node]["data_max"]])
        push!(batch_info[node], "lower" => value)
        push!(batch_info[node], "upper" => value)
        push!(batch_info[node], "bound" => new_bound)
    end
end


In [ ]:
model = Chain([
    Flux.flatten,
    Dense(784, 200, relu),
    Dense(200, 10)
])
image_seeds = [MNIST(:train)[i].features for i in 1:1]
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/debug.onnx"
Flux_model = model
image_shape = (28, 28, 1, 1)